# MM4 e Calcolo Risk Premia
Questo codice deve essere lanciato due volte, una con le componenti principali costruite con le variabili grezze e una con le componenti principali costruite con gli shock.

In [2]:
import pickle
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
import statistics
import scipy.stats as stats


## Import Alpha di Fama e French Five Factors

In [ ]:
countries = ["Germania", "Francia", "Italia", "Spagna", "Finlandia", "Olanda"]
file_names = ["a_SizeBM", "a_SizeEbit", "a_SizeINV"]

# Percorso dei file
base_folder = "/Users/nome/****/****"

alpha = {}
Alpha = {} # Sarà utile poi per calcolare i fattori macroeconomici.

# Definisci i nomi dei portafogli.
portfolio_names = [f"Portafoglio_{i}" for i in range(1, 10)]

# Itera su ogni paese e file
for country in countries:
    for file_name in file_names:
        file_path = f"{base_folder}/{country}/****/{file_name}.pkl"
        
        if os.path.exists(file_path):
            df = pd.read_pickle(file_path)
            print(f"Dati caricati per {country} - {file_name}.")
            Alpha[f"{country}_{file_name}"] = df

            # Si suddividono gli alpha per portafogli.
            a = {}
            for year, quarters in df.items():  
                a[year] = {}  

                for quarter, values in quarters.items():
                    a[year][quarter] = dict(zip(portfolio_names, values))
            
            alpha[f"{country}_{file_name}"] = a
        else:
            print(f"File non trovato: {file_path}")

# Esempio di accesso ai dati rinominati
print(alpha['Germania_a_SizeBM']['anno_1']['Q1']['Portafoglio_1'])


Si calcolano per ogni portafoglio le medie degli alpha a livello europeo nei 20 anni e si visualizzano (facoltativo).

In [ ]:
# Dizionario per accumulare i dati delle nazioni e calcolare la media
alpha['Europa'] = defaultdict(lambda: defaultdict(lambda: defaultdict(dict)))

for file_key in file_keys:
    for year in alpha[f'{countries[0]}_{file_key}']: 
        for quarter in alpha[f'{countries[0]}_{file_key}'][year]: 
            
            # Media per ogni portafoglio
            portfolio_sums = defaultdict(float)
            count = 0

            # Somma i valori di ogni portafoglio per ogni paese
            for country in countries:
                country_key = f"{country}_{file_key}"
                
                if quarter in alpha[country_key][year]:
                    for portfolio, value in alpha[country_key][year][quarter].items():
                        portfolio_sums[portfolio] += value
                    count += 1

            # Calcola la media per ogni portafoglio e aggiungila ai dati di Europa
            for portfolio, total in portfolio_sums.items():
                if count > 0:
                    alpha['Europa'][file_key][year][quarter][portfolio] = total / count
                    

In [ ]:
portfolios = [f'Portafoglio_{i+1}' for i in range(9)]
categories = ['a_SizeBM', 'a_SizeEbit', 'a_SizeINV']
years = [f'anno_{i+1}' for i in range(1, 21)]

# Funzione per calcolare la media degli alpha su 20 anni e creare la heatmap
def heatmap_alpha_medi_su_20_anni(alpha_data, category, title):
    alpha_medi_20_anni = {}

    # Calcola la media annuale per ogni portafoglio per la categoria
    for portfolio in portfolios:
        annual_means = []
        for year in years:
            quarterly_values = [
                alpha_data['Europa'][category][year].get(quarter, {}).get(portfolio, None)
                for quarter in ['Q1', 'Q2', 'Q3', 'Q4']
            ]
            quarterly_values = [v for v in quarterly_values if v is not None]
            if quarterly_values:
                annual_means.append(sum(quarterly_values) / len(quarterly_values))
        
        # Calcola la media dei 20 anni
        if annual_means:
            alpha_medi_20_anni[portfolio] = sum(annual_means) / len(annual_means)
    
    # Converti i dati in un DataFrame per la heatmap
    alpha_medi_20_anni_df = pd.DataFrame([alpha_medi_20_anni], index=[category])

    # Crea la heatmap
    plt.figure(figsize=(12, 2))
    sns.heatmap(alpha_medi_20_anni_df, annot=True, fmt=".2f", cmap="coolwarm", cbar_kws={'label': 'Valore Medio Alpha'})
    plt.title(title, fontsize=16, fontweight="bold")
    plt.xlabel("Portafoglio")
    plt.ylabel("")
    plt.show()

# Genera le heatmap per ciascuna categoria
for category in categories:
    heatmap_alpha_medi_su_20_anni(alpha, category, f"Heatmap degli Alpha Medi Aggregati - {category}")


## Import degli Indicatori Macroeconomici

In [ ]:
file_names = ["pc1_shock_all_nations", "pc2_shock_all_nations", "pc3_shock_all_nations", "pc4_shock_all_nations"]

# Cartella base del percorso
base_folder = "/Users/micha/****/****/****"

Shock = {}

for file_name in file_names:
    file_path = f"{base_folder}/{file_name}.pkl"
    if os.path.exists(file_path):
        df = pd.read_pickle(file_path)
        print(df)
        df = df.dropna()
        Shock[f"{file_name}"] = df
        print(f"Dati caricati per {file_name}.")
    else:
        print(f"File non trovato: {file_path}")
        

Ristrutturazione del dizionario.

In [ ]:
# Funzione per ristrutturare i dati
def restructure_data(shock_data):
    # Inizializza il dizionario principale per Shock
    Shock_final = {
        'PC1': {},
        'PC2': {},
        'PC3': {},
        'PC4': {},
    }
    
    # Estrae i dati di produzione, spread e inflazione
    PC1 = shock_data['pc1_shock_all_nations']
    PC2 = shock_data['pc2_shock_all_nations']
    PC3 = shock_data['pc3_shock_all_nations']
    PC4 = shock_data['pc4_shock_all_nations']
    
    def extract_data(data, category_name):
        """Funzione interna per estrarre i dati trimestrali."""
        for country in data.columns:
            country_years = {}
            values = data[country].values
            
            for i in range(0, len(values), 4):
                year_index = i // 4 + 1  
                year_key = f'anno_{year_index}'
                
                # Crea un dizionario per l'anno 
                if year_key not in country_years:
                    country_years[year_key] = {
                        'Q3': [],  
                        'Q4': [],  
                        'Q1': [],  
                        'Q2': []  
                    }
                
                # Aggiunge i dati ai trimestri
                if i < len(values):  # Q3
                    country_years[year_key]['Q3'].append(values[i])
                if i + 1 < len(values):  # Q4
                    country_years[year_key]['Q4'].append(values[i + 1])
                if i + 2 < len(values):  # Q1
                    country_years[year_key]['Q1'].append(values[i + 2])
                if i + 3 < len(values):  # Q2
                    country_years[year_key]['Q2'].append(values[i + 3])
            
            # Aggiunge i dati della nazione al dizionario finale
            Shock_final[category_name][country] = country_years


    # Estrai i dati per ogni categoria
    extract_data(PC1, 'PC1')
    extract_data(PC2, 'PC2')
    extract_data(PC3, 'PC3')
    extract_data(PC4, 'PC4')

    return Shock_final

# Creia il nuovo dizionario Shock
restructured_shock = restructure_data(Shock)


## Calcolo dei Fattori Macroeconomici
Si calcola per ogni periodo la media degli alpha tra tutti i portafogli di ogni nazione.

In [ ]:
import numpy as np

# Dizionario per contenere le medie
media_alpha = {}

# Elenco delle nazioni e portafogli
nazioni = ['Germania', 'Francia', 'Italia', 'Spagna', 'Finlandia', 'Olanda']
portafogli = ['SizeBM', 'SizeEbit', 'SizeINV']

# Per ogni nazione
for nazione in nazioni:
    media_alpha[nazione] = {}

    # Per ogni anno
    for anno in range(1, 21):  
        anno_key = f'anno_{anno}'
        media_alpha[nazione][anno_key] = {}
        
        rendimenti_Q3 = []
        rendimenti_Q4 = []
        rendimenti_Q1 = []
        rendimenti_Q2 = []

        # Per ogni categoria di portafoglio (es. SizeBM)
        for size in portafogli:
            portafoglio_key = f'{nazione}_a_{size}'  
            if portafoglio_key in alpha:

                # Per ogni portafoglio
                for n in range(1,10):
                    portafoglio = f'Portafoglio_{n}'
                # Accumula i rendimenti per i trimestri
                rendimenti_Q3.append(alpha[portafoglio_key][anno_key]['Q3'][portafoglio])
                rendimenti_Q4.append(alpha[portafoglio_key][anno_key]['Q4'][portafoglio])
                rendimenti_Q1.append(alpha[portafoglio_key][anno_key]['Q1'][portafoglio])
                rendimenti_Q2.append(alpha[portafoglio_key][anno_key]['Q2'][portafoglio])

        # Calcola la media tra i portafogli di tutte le categorie di ogni nazione per ogni trimestre
        media_alpha[nazione][anno_key]['Q3'] = np.mean(rendimenti_Q3) if rendimenti_Q3 else None
        media_alpha[nazione][anno_key]['Q4'] = np.mean(rendimenti_Q4) if rendimenti_Q4 else None
        media_alpha[nazione][anno_key]['Q1'] = np.mean(rendimenti_Q1) if rendimenti_Q1 else None
        media_alpha[nazione][anno_key]['Q2'] = np.mean(rendimenti_Q2) if rendimenti_Q2 else None
        

Si trovano per ogni periodo i paesi con componente principale sopra e sotto alla mediana.

In [ ]:
# Funzione per calcolare "above" e "below" 
def calcola_above_below(variabili):
    # Ordina i paesi in base ai valori e seleziona i primi 3 e gli ultimi 3
    ordinato = sorted(variabili, key=lambda x: x[1], reverse=True)  
    above = [x[0] for x in ordinato[:3]]  
    below = [x[0] for x in ordinato[3:]]  
    return above, below

# Dizionari per memorizzare i fattori
PC1_factors = {year: {} for year in range(1, 21)}
PC2_factors = {year: {} for year in range(1, 21)}
PC3_factors = {year: {} for year in range(1, 21)}
PC4_factors = {year: {} for year in range(1, 21)}

# Dizionari per il calcolo delle statistiche
above_counts = {'PC1': {}, 'PC2': {}, 'PC3': {}, 'PC4': {}}
below_counts = {'PC1': {}, 'PC2': {}, 'PC3': {}, 'PC4': {}}

# Per ogni anno
for year in range(1, 21):
    # Per ogni trimestre
    for quarter in ['Q3', 'Q4', 'Q1', 'Q2']:
        pc1_values = []
        pc2_values = []
        pc3_values = []
        pc4_values = []
        # Per ogni paese
        for country in restructured_shock['PC1']:
            pc1_value = restructured_shock['PC1'][country][f'anno_{year}'][quarter][0]
            pc1_values.append((country, pc1_value))

            pc2_value = restructured_shock['PC2'][country][f'anno_{year}'][quarter][0]
            pc2_values.append((country, pc2_value))

            pc3_value = restructured_shock['PC3'][country][f'anno_{year}'][quarter][0]
            pc3_values.append((country, pc3_value))

            pc4_value = restructured_shock['PC4'][country][f'anno_{year}'][quarter][0]
            pc4_values.append((country, pc4_value))

        # Calcolare i paesi sopra e sotto la mediana
        above_pc1, below_pc1 = calcola_above_below(pc1_values)
        above_pc2, below_pc2 = calcola_above_below(pc2_values)
        above_pc3, below_pc3 = calcola_above_below(pc3_values)
        above_pc4, below_pc4 = calcola_above_below(pc4_values)

        # Aggiorna i conteggi per ciascun paese
        for country in above_pc1:
            above_counts['PC1'][country] = above_counts['PC1'].get(country, 0) + 1
        for country in below_pc1:
            below_counts['PC1'][country] = below_counts['PC1'].get(country, 0) + 1

        for country in above_pc2:
            above_counts['PC2'][country] = above_counts['PC2'].get(country, 0) + 1
        for country in below_pc2:
            below_counts['PC2'][country] = below_counts['PC2'].get(country, 0) + 1

        for country in above_pc3:
            above_counts['PC3'][country] = above_counts['PC3'].get(country, 0) + 1
        for country in below_pc3:
            below_counts['PC3'][country] = below_counts['PC3'].get(country, 0) + 1

        for country in above_pc4:
            above_counts['PC4'][country] = above_counts['PC4'].get(country, 0) + 1
        for country in below_pc4:
            below_counts['PC4'][country] = below_counts['PC4'].get(country, 0) + 1

        # Calcolo dei fattori di PC1, PC2, PC3, PC4 per il trimestre e l'anno corrente
        # I fattori sono la differenza tra la media dei paesi "above" e "below" 
        avg_above_pc1 = np.mean([media_alpha[country][f'anno_{year}'][quarter] for country in above_pc1]) if above_pc1 else 0
        avg_below_pc1 = np.mean([media_alpha[country][f'anno_{year}'][quarter] for country in below_pc1]) if below_pc1 else 0
        PC1_factors[year][quarter] = avg_above_pc1 - avg_below_pc1

        avg_above_pc2 = np.mean([media_alpha[country][f'anno_{year}'][quarter] for country in above_pc2]) if above_pc2 else 0
        avg_below_pc2 = np.mean([media_alpha[country][f'anno_{year}'][quarter] for country in below_pc2]) if below_pc2 else 0
        PC2_factors[year][quarter] = avg_above_pc2 - avg_below_pc2

        avg_above_pc3 = np.mean([media_alpha[country][f'anno_{year}'][quarter] for country in above_pc3]) if above_pc3 else 0
        avg_below_pc3 = np.mean([media_alpha[country][f'anno_{year}'][quarter] for country in below_pc3]) if below_pc3 else 0
        PC3_factors[year][quarter] = avg_above_pc3 - avg_below_pc3

        avg_above_pc4 = np.mean([media_alpha[country][f'anno_{year}'][quarter] for country in above_pc4]) if above_pc4 else 0
        avg_below_pc4 = np.mean([media_alpha[country][f'anno_{year}'][quarter] for country in below_pc4]) if below_pc4 else 0
        PC4_factors[year][quarter] = avg_above_pc4 - avg_below_pc4


# Stampa delle statistiche
print("Above Median Counts:")
for variable, counts in above_counts.items():
    print(f"{variable}: {counts}")

print("\nBelow Median Counts:")
for variable, counts in below_counts.items():
    print(f"{variable}: {counts}")


# Modello MM4 

Si iniziaizza il dataframe che conterrà variabile dipendente e fattori.

In [ ]:
from statsmodels.stats.diagnostic import het_white
from statsmodels.stats.stattools import jarque_bera
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm

## Regressioni

In [ ]:
data_frames = {}

paesi = ['Germania', 'Francia', 'Italia', 'Spagna', 'Finlandia', 'Olanda']

for paese in paesi:
    data_frames[paese] = {} 
    categorie = [f'{paese}_a_SizeBM', f'{paese}_a_SizeEbit', f'{paese}_a_SizeINV']
    
    # Per ogni categoria (es. SizeBM)
    for categoria in categorie:
        categoria_base = categoria.split('_')[-1]
        
        # Per ogni portafoglio.
        for i in range(1, 10):
            portafoglio_key = f'{categoria_base}_portafoglio_{i}'
            
            y = []
            PC1 = []
            PC2 = []
            PC3 = []
            PC4 = []

            # Per ogni anno e trimestre:
            for anno, trimestri in alpha[categoria].items():
                for trimestre, portafogli in trimestri.items():
                    # estrai il valore alpha per il portafoglio attuale e aggiungilo alla serie y
                    y.append(portafogli[f'Portafoglio_{i}'])
                    
                    PC1.append(PC1_factors[int(anno.split('_')[1])][trimestre])
                    PC2.append(PC2_factors[int(anno.split('_')[1])][trimestre])
                    PC3.append(PC3_factors[int(anno.split('_')[1])][trimestre])
                    PC4.append(PC4_factors[int(anno.split('_')[1])][trimestre])
            
            # Aggiungi la serie y e i regressori al dizionario data_frames
            data_frames[paese][portafoglio_key] = {
                'y': y,
                'PC1_factors': PC1,
                'PC2_factors': PC2,
                'PC3_factors': PC3,
                'PC4_factors': PC4,

            }

Di seguito il codice per eseguire una regressione per ogni portafoglio scegliendo sulla base dei vari test quale modello utilizzare (vedere tesi). Si salvano i risultati e si printano le statistiche.

In [ ]:
# Dizionario per memorizzare i risultati di ogni regressione per ciascun paese
regression_results = {}

# Contatori per le statistiche
total_regressions_saved = 0
non_normal_residuals_count = 0
heteroschedastic_residuals_count = 0
ols_count = 0
ols_hc3_count = 0
rlm_count = 0

# Contatori per significatività
ols_significant_count = 0
ols_hc3_significant_count = 0
rlm_significant_count = 0

# Contatori per multicollinearità
multicollinearity_count = 0

# Liste per accumulare gli R^2
ols_r_squared_values = []
ols_hc3_r_squared_values = []
rlm_pseudo_r_squared_values = []

# Per ogni paese
for paese in paesi:
    regression_results[paese] = {}  
    categorie = ['SizeBM', 'SizeEbit', 'SizeINV']
    
    # Per ogni categoria
    for categoria in categorie:
        # Per ogni portafoglio
        for i in range(1, 10):
            portafoglio_key = f'{categoria}_portafoglio_{i}'
            
            y = pd.Series(data_frames[paese][portafoglio_key]['y'])
            x1 = pd.Series(data_frames[paese][portafoglio_key]['PC1_factors'])
            x2 = pd.Series(data_frames[paese][portafoglio_key]['PC2_factors'])
            x3 = pd.Series(data_frames[paese][portafoglio_key]['PC3_factors'])
            x4 = pd.Series(data_frames[paese][portafoglio_key]['PC4_factors'])

            X_poly = pd.DataFrame({
                'PC1': x1,
                'PC2': x2,
                'PC3': x3,
                'PC4': x4
            })

            # Aggiungo l'intercetta
            X_poly = sm.add_constant(X_poly)

            # Calcolo VIF per ogni regressore
            vif_data = pd.DataFrame()
            vif_data["Variable"] = X_poly.columns
            vif_data["VIF"] = [variance_inflation_factor(X_poly.values, i) for i in range(X_poly.shape[1])]

            # Se c'è multicollinearità (VIF > 10), incremento il contatore
            if (vif_data["VIF"] > 10).any():
                multicollinearity_count += 1
                print(f"Multicollinearità rilevata in {paese} per {portafoglio_key} (VIF > 10)")
            
            # Esegui la regressione OLS
            ols_model = sm.OLS(y, X_poly).fit()
            
            # Test di Jarque-Bera per la normalità dei residui
            jb_test = jarque_bera(ols_model.resid)
            jb_pvalue = jb_test[1] 

            is_significant = False  

            if jb_pvalue < 0.05:                # Se non normalità uso RLM
                non_normal_residuals_count += 1

                rlm_model = sm.RLM(y, X_poly).fit()
                rlm_count += 1
                total_regressions_saved += 1

                ss_total = np.sum((y - np.mean(y))**2)  
                ss_resid = np.sum(rlm_model.resid**2)
                pseudo_r_squared = 1 - (ss_resid / ss_total)
                

                # Controlla la significatività del modello RLM (basato sui p-value dei coefficienti)
                if rlm_model.pvalues.iloc[1:].min() < 0.05:  
                    rlm_significant_count += 1
                    is_significant = True
                coefficients = pd.Series(rlm_model.params, index=X_poly.columns)
                pvalues = pd.Series(rlm_model.pvalues, index=X_poly.columns)
                
                regression_results[paese][portafoglio_key] = {
                    'coefficients': coefficients,
                    'pvalues': pvalues,
                    'r_squared': pseudo_r_squared,
                    'is_significant': is_significant
                }
            else:
                # Test di White per eteroschedasticità
                white_test = het_white(ols_model.resid, ols_model.model.exog)
                white_pvalue = white_test[1]

                if white_pvalue < 0.05:                 # Se c'è eteroschedasticità uso OLS HC3
                    heteroschedastic_residuals_count += 1

                    ols_hc3_model = ols_model.get_robustcov_results(cov_type='HC3')
                    ols_hc3_count += 1
                    total_regressions_saved += 1

                    ols_hc3_r_squared_values.append(ols_hc3_model.rsquared)

                    # Controlla la significatività del modello OLS HC3 (F-statistic)
                    if ols_hc3_model.f_pvalue < 0.05:  
                        ols_hc3_significant_count += 1
                        is_significant = True

                    coefficients = pd.Series(ols_hc3_model.params, index=X_poly.columns)
                    pvalues = pd.Series(ols_hc3_model.pvalues, index=X_poly.columns)

                    regression_results[paese][portafoglio_key] = {
                        'coefficients': coefficients,#ols_hc3_model.params,
                        'pvalues': pvalues,#ols_hc3_model.pvalues,
                        'r_squared': ols_hc3_model.rsquared,
                        'is_significant': is_significant
                    }
                else:
                    # Usa OLS standard
                    ols_count += 1
                    total_regressions_saved += 1

                    ols_r_squared_values.append(ols_model.rsquared)

                    # Controlla la significatività del modello OLS standard (F-statistic)
                    if ols_model.f_pvalue < 0.05: 
                        ols_significant_count += 1
                        is_significant = True

                    regression_results[paese][portafoglio_key] = {
                        'coefficients': ols_model.params,
                        'pvalues': ols_model.pvalues,
                        'r_squared': ols_model.rsquared,
                        'is_significant': is_significant
                    }

# Calcola gli R^2 medi
ols_r_squared_mean = np.mean(ols_r_squared_values) if ols_r_squared_values else np.nan
ols_hc3_r_squared_mean = np.mean(ols_hc3_r_squared_values) if ols_hc3_r_squared_values else np.nan
rlm_pseudo_r_squared_mean = np.mean(rlm_pseudo_r_squared_values) if rlm_pseudo_r_squared_values else np.nan

# Stampa del riepilogo
print(f"Totale regressioni salvate: {total_regressions_saved}")
print(f"Totale regressioni con residui non normali (Jarque-Bera): {non_normal_residuals_count}")
print(f"Totale regressioni con residui eteroschedastici (White test): {heteroschedastic_residuals_count}")
print(f"Regressioni eseguite con OLS: {ols_count}")
print(f"Regressioni eseguite con OLS HC3: {ols_hc3_count}")
print(f"Regressioni eseguite con RLM: {rlm_count}")
print(f"Regressioni OLS significative (F-statistic < 0.05): {ols_significant_count}")
print(f"Regressioni OLS HC3 significative (F-statistic < 0.05): {ols_hc3_significant_count}")
print(f"Regressioni RLM significative (coefficients p-value < 0.05): {rlm_significant_count}")

print("\n--- R² Medi ---")
print(f"R² medio per OLS: {ols_r_squared_mean:.4f}" if not np.isnan(ols_r_squared_mean) else "Nessun R² calcolato per OLS.")
print(f"R² medio per OLS HC3: {ols_hc3_r_squared_mean:.4f}" if not np.isnan(ols_hc3_r_squared_mean) else "Nessun R² calcolato per OLS HC3.")
print(f"Pseudo R² medio per RLM: {rlm_pseudo_r_squared_mean:.4f}" if not np.isnan(rlm_pseudo_r_squared_mean) else "Nessun pseudo R² calcolato per RLM.")


## Analisi R quadri (Facoltativo)

Visualizzazione degli R quadri e pseudo R quadri medi.

In [ ]:
# Funzione per calcolare gli R^2 aggiustati e creare heatmap
def calcola_e_visualizza_r2_adj(regression_results, tipo_size, paesi):
    r2_adj_data = {f'Portafoglio_{i}': [] for i in range(1, 10)}

    for paese in paesi:
        for i in range(1, 10):
            portafoglio_key = f'{tipo_size}_portafoglio_{i}'

            if paese in regression_results and portafoglio_key in regression_results[paese]:
                r2 = regression_results[paese][portafoglio_key].get('r_squared', None)

                # Se il valore di r2_adj è valido (non None), aggiungilo alla lista
                if r2 is not None:
                    r2_data[f'Portafoglio_{i}'].append(r2)
                else:
                    r2_data[f'Portafoglio_{i}'].append(None)  # Aggiungi None se 'r_squared' è mancante
            else:
                r2_data[f'Portafoglio_{i}'].append(None)

    # Crea il DataFrame con l'indice dei Paesi e le colonne per i portafogli
    r2_df = pd.DataFrame(r2_data, index=paesi)

    r2_df.loc['Media'] = r2_df.mean()

    r2_df['Media'] = r2_df.mean(axis=1)

    # Converti i valori dell'R^2 aggiustato in percentuale
    r2_df_percentage = r2_df * 100 

    plt.figure(figsize=(10, 6))  
    sns.heatmap(r2_df_percentage, annot=True, fmt=".1f", cmap="YlGnBu", cbar_kws={'label': 'R^2 Aggiustato (%)'})
    plt.title(f"R^2 Aggiustato (%) per ciascun portafoglio {tipo_size} e paese")
    plt.xlabel(f"Portafogli {tipo_size}")
    plt.ylabel("Paesi")
    plt.show()


calcola_e_visualizza_r2_adj(regression_results, 'SizeBM', paesi)
calcola_e_visualizza_r2_adj(regression_results, 'SizeEbit', paesi)
calcola_e_visualizza_r2_adj(regression_results, 'SizeINV', paesi)


Calcolo degli R quadri medi per ogni categoria.

In [ ]:
import statistics

paesi = ['Germania','Francia','Italia','Spagna', 'Finlandia','Olanda']
categorie = ['SizeBM', 'SizeEbit', 'SizeINV']

r_quadro_small = []
r_quadro_mid = []
r_quadro_big = []

small_all = []
mid_all = []
big_all = []

for paese in paesi:
    small = []
    mid = []
    big = []
    for i in [1,2,3,4,5,6,7,8,9]:
        if i in [1,2,3]:
            for categoria in categorie:
                x = regression_results[paese][f'{categoria}_portafoglio_{i}']['r_squared']
                small.append(x)
                small_all.append(x)
                
        if i in [4,5,6]:
            for categoria in categorie:
                x = regression_results[paese][f'{categoria}_portafoglio_{i}']['r_squared']
                mid.append(x)
                mid_all.append(x)

        if i in [7,8,9]:
            for categoria in categorie:
                x = regression_results[paese][f'{categoria}_portafoglio_{i}']['r_squared']
                big.append(x)
                big_all.append(x)
                
    media_small = statistics.mean(small)
    media_mid = statistics.mean(mid)
    media_big = statistics.mean(big)

    r_quadro_small.append(media_small)
    r_quadro_mid.append(media_mid)
    r_quadro_big.append(media_big)

print('R quadro medio delle small per ogni paese:', len(r_quadro_small))
print('R quadro medio delle mid per ogni paese:', len(r_quadro_mid))
print('R quadro medio delle big per ogni paese:', len(r_quadro_big))

r_quadro_low = []
r_quadro_mean = []
r_quadro_high = []

low_all = []
mean_all = []
high_all = []

for paese in paesi:
    low = []
    mean = []
    high = []
    for i in [1,2,3,4,5,6,7,8,9]:
        if i in [1,4,7]:
            x = regression_results[paese][f'SizeBM_portafoglio_{i}']['r_squared']
            low.append(x)
            low_all.append(x)
        elif i in [2,5,8]:
            x = regression_results[paese][f'SizeBM_portafoglio_{i}']['r_squared']
            mean.append(x)
            mean_all.append(x)
        else:
            x = regression_results[paese][f'SizeBM_portafoglio_{i}']['r_squared']
            high.append(x)
            high_all.append(x)

    media_low = statistics.mean(low)
    media_mean = statistics.mean(mean)
    media_high = statistics.mean(high)

    r_quadro_low.append(media_low)
    r_quadro_mean.append(media_mean)
    r_quadro_high.append(media_high)

print('R quadro medio delle low per ogni paese:', len(r_quadro_low))
print('R quadro medio delle mean per ogni paese:', len(r_quadro_mean))
print('R quadro medio delle high per ogni paese:', len(r_quadro_high))

r_quadro_weak = []
r_quadro_mean1 = []
r_quadro_rob = []

weak_all = []
mean1_all = []
rob_all = []

for paese in paesi:
    weak = []
    mean1 = []
    rob = []
    for i in [1,2,3,4,5,6,7,8,9]:
        if i in [1,4,7]:
            x = regression_results[paese][f'SizeEbit_portafoglio_{i}']['r_squared']
            weak.append(x)
            weak_all.append(x)
        if i in [2,5,8]:
            x = regression_results[paese][f'SizeEbit_portafoglio_{i}']['r_squared']
            mean1.append(x)
            mean1_all.append(x)
        if i in [3,6,9]:
            x = regression_results[paese][f'SizeEbit_portafoglio_{i}']['r_squared']
            rob.append(x)
            rob_all.append(x)

    media_weak = statistics.mean(weak)
    media_mean1 = statistics.mean(mean1)
    media_rob = statistics.mean(rob)

    r_quadro_weak.append(media_weak)
    r_quadro_mean1.append(media_mean1)
    r_quadro_rob.append(media_rob)

print('R quadro medio delle low per ogni paese:', len(r_quadro_weak))
print('R quadro medio delle mean per ogni paese:', len(r_quadro_mean1))
print('R quadro medio delle high per ogni paese:', len(r_quadro_rob))

r_quadro_cons = []
r_quadro_mean2 = []
r_quadro_aggr = []

cons_all = []
mean2_all = []
aggr_all = []

for paese in paesi:
    cons = []
    mean2 = []
    aggr = []
    for i in [1,2,3,4,5,6,7,8,9]:
        if i in [1,4,7]:
            x = regression_results[paese][f'SizeINV_portafoglio_{i}']['r_squared']
            cons.append(x)
            cons_all.append(x)
        if i in [2,5,8]:
            x = regression_results[paese][f'SizeINV_portafoglio_{i}']['r_squared']
            mean2.append(x)
            mean2_all.append(x)
        if i in [3,6,9]:
            x = regression_results[paese][f'SizeINV_portafoglio_{i}']['r_squared']
            aggr.append(x)
            aggr_all.append(x)

    media_cons = statistics.mean(cons)
    media_mean2 = statistics.mean(mean2)
    media_aggr = statistics.mean(aggr)

    r_quadro_cons.append(media_cons)
    r_quadro_mean2.append(media_mean2)
    r_quadro_aggr.append(media_aggr)

print('R quadro medio delle low per ogni paese:', len(r_quadro_cons))
print('R quadro medio delle mean per ogni paese:', len(r_quadro_mean2))
print('R quadro medio delle high per ogni paese:', len(r_quadro_aggr))

Preparazione del Dataset per la creazione di grafici.

In [ ]:
sizes = ['Small Cap', 'Mid Cap', 'Big Cap', 'Small Cap', 'Mid Cap', 'Big Cap', 'Small Cap', 'Mid Cap', 'Big Cap',
        'Small Cap', 'Mid Cap', 'Big Cap', 'Small Cap', 'Mid Cap', 'Big Cap', 'Small Cap', 'Mid Cap', 'Big Cap',]

Nazione = ['Germania', 'Germania', 'Germania', 'Francia', 'Francia', 'Francia', 'Italia', 'Italia', 'Italia',
           'Spagna', 'Spagna', 'Spagna', 'Finlandia', 'Finlandia', 'Finlandia', 'Olanda', 'Olanda', 'Olanda']

df4 = pd.DataFrame({
    'Small': small_all,
    'Mid': mid_all,
    'Big': big_all
})

df = pd.DataFrame({
    'Category': sizes, 
    'Low': low_all,
    #'Mean': mean_all,
    'High': high_all
})

df1 = pd.DataFrame({
    'Category': sizes, 
    'Weak': weak_all,
    #'Mean': mean1_all,
    'Robust': rob_all
})

df2 = pd.DataFrame({
    'Category': sizes, 
    'Conservative': cons_all,
    #'Mean': mean2_all,
    'Aggressive': aggr_all
})
df = df[~df.isin(['Mid Cap']).any(axis=1)]
df1 = df1[~df1.isin(['Mid Cap']).any(axis=1)]
df2 = df2[~df2.isin(['Mid Cap']).any(axis=1)]
df_melted = df4.melt(value_vars=["Small","Mid", "Big"], 
                    var_name="Type", value_name="Value")

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(20, 6))

df_melted = df4.melt(value_vars=["Small","Mid", "Big"], 
                    var_name="Type", value_name="Value")

sns.violinplot(y='Value', hue='Type', data=df_melted, inner="quart", 
               palette="muted", ax=axes[0], alpha=0.75)
axes[0].set_title('Violin Plot per Small, Mid e Big Cap')
axes[0].set_ylabel('Valore')

df_melted = df.melt(id_vars=["Category"], value_vars=["Low", "High"], 
                    var_name="Type", value_name="Value")

sns.violinplot(x='Category', y='Value', hue='Type', data=df_melted, 
               inner="quart", palette="muted", split=True, ax=axes[1], alpha=0.75)
axes[1].set_title('Violin Plot per Low, Mean e High')
axes[1].set_ylabel('Valore')

df_melted = df1.melt(id_vars=["Category"], value_vars=["Weak","Robust"], 
                    var_name="Type", value_name="Value")

sns.violinplot(x='Category', y='Value', hue='Type', data=df_melted, 
               inner="quart", palette="muted", split=True, ax=axes[2], alpha=0.75)
axes[2].set_title('Violin Plot per Weak, Mean e Robust')
axes[2].set_ylabel('Valore')

df_melted = df2.melt(id_vars=["Category"], value_vars=["Conservative", "Aggressive"], 
                    var_name="Type", value_name="Value")

sns.violinplot(x='Category', y='Value', hue='Type', data=df_melted, 
               inner="quart", palette="muted", split=True, ax=axes[3], alpha=0.75)
axes[3].set_title('Violin Plot per Cons, Mean e Aggr')
axes[3].set_ylabel('Valore')

plt.tight_layout()
plt.show()


Test di Kruskal-Wallis per testare la differenza tra i gruppi.

In [ ]:
# Differenza tra Small e Big
stat, p_value_kw = stats.kruskal(small_all, big_all)

print(f"Test di Kruskal-Wallis statistic: {stat:.2f}")
print(f"P-value: {p_value_kw:.4f}")

if p_value_kw < 0.05:
    print("Le medie tra i gruppi sono significativamente diverse (p < 0.05).")
else:
    print("Le medie tra i gruppi non sono significativamente diverse (p >= 0.05).")

# Differenza tra Low e High
f_stat, p_value = stats.kruskal(low_all,  high_all)

print(f"F-statistic: {f_stat:.2f}")
print(f"P-value: {p_value:.4f}")

if p_value < 0.05:
    print("Le medie sono significativamente diverse tra loro (p < 0.05).")
else:
    print("Le medie non sono significativamente diverse tra loro (p >= 0.05).")

# Differenza tra Weak e Robust
f_stat, p_value = stats.kruskal(weak_all,  rob_all)

print(f"F-statistic: {f_stat:.2f}")
print(f"P-value: {p_value:.4f}")

if p_value < 0.05:
    print("Le medie sono significativamente diverse tra loro (p < 0.05).")
else:
    print("Le medie non sono significativamente diverse tra loro (p >= 0.05).")

# Differenza tra Conservative e Aggressive
f_stat, p_value = stats.kruskal(cons_all, aggr_all)

print(f"F-statistic: {f_stat:.2f}")
print(f"P-value: {p_value:.4f}")

if p_value < 0.05:
    print("Le medie sono significativamente diverse tra loro (p < 0.05).")
else:
    print("Le medie non sono significativamente diverse tra loro (p >= 0.05).")

## Analisi dei Coefficienti (facoltativo)

In [ ]:
coefficiente = 4

coeff_1_small = []
coeff_2_small = []
coeff_3_small = []
coeff_4_small = []
coeff_1_mid = []
coeff_2_mid = []
coeff_3_mid = []
coeff_4_mid = []
coeff_1_big = []
coeff_2_big = []
coeff_3_big = []
coeff_4_big = []

small_coeff_1 = []
mid_coeff_1 = []
big_coeff_1 = []
small_coeff_2 = []
mid_coeff_2 = []
big_coeff_2 = []
small_coeff_3 = []
mid_coeff_3 = []
big_coeff_3 = []
small_coeff_4 = []
mid_coeff_4 = []
big_coeff_4 = []

for paese in paesi:
    small_1 = []
    small_2 = []
    small_3 = []
    small_4 = []
    mid_1 = []
    mid_2 = []
    mid_3 = []
    mid_4 = []
    big_1 = []
    big_2 = []
    big_3 = []
    big_4 = []
    for i in [1,2,3,4,5,6,7,8,9]:
        if i in [1,2,3]:
            for categoria in categorie:
                for n in [1,2,3,4]:
                    x = regression_results[paese][f'{categoria}_portafoglio_{i}']['coefficients'][n]
                    if n == 1:
                        small_1.append(x)
                        small_coeff_1.append(x)
                    elif n == 2:
                        small_2.append(x)
                        small_coeff_2.append(x)
                    elif n == 3:
                        small_3.append(x)
                        small_coeff_3.append(x)
                    else:
                        small_4.append(x)
                        small_coeff_4.append(x)
                
        if i in [4,5,6]:
            for categoria in categorie:
                for n in [1,2,3,4]:
                    x = regression_results[paese][f'{categoria}_portafoglio_{i}']['coefficients'][n]
                    if n == 1:
                        mid_1.append(x)
                        mid_coeff_1.append(x)
                    elif n == 2:
                        mid_2.append(x)
                        mid_coeff_2.append(x)
                    elif n == 3:
                        mid_3.append(x)
                        mid_coeff_3.append(x)
                    else:
                        mid_4.append(x)
                        mid_coeff_4.append(x)
        if i in [7,8,9]:
            for categoria in categorie:
                for n in [1,2,3,4]:
                    x = regression_results[paese][f'{categoria}_portafoglio_{i}']['coefficients'][n]
                    if n == 1:
                        big_1.append(x)
                        big_coeff_1.append(x)
                    elif n == 2:
                        big_2.append(x)
                        big_coeff_2.append(x)
                    elif n == 3:
                        big_3.append(x)
                        big_coeff_3.append(x)
                    else:
                        big_4.append(x)
                        big_coeff_4.append(x)
                
    media_small_1 = statistics.mean(small_1)
    media_small_2 = statistics.mean(small_2)
    media_small_3 = statistics.mean(small_3)
    media_small_4 = statistics.mean(small_4)
    media_mid_1 = statistics.mean(mid_1)
    media_mid_2 = statistics.mean(mid_2)
    media_mid_3 = statistics.mean(mid_3)
    media_mid_4 = statistics.mean(mid_4)
    media_big_1 = statistics.mean(big_1)
    media_big_2 = statistics.mean(big_2)
    media_big_3 = statistics.mean(big_3)
    media_big_4 = statistics.mean(big_4)

    coeff_1_small.append(media_small_1)
    coeff_2_small.append(media_small_2)
    coeff_3_small.append(media_small_3)
    coeff_4_small.append(media_small_4)
    coeff_1_mid.append(media_mid_1)
    coeff_2_mid.append(media_mid_2)
    coeff_3_mid.append(media_mid_3)
    coeff_4_mid.append(media_mid_4)
    coeff_1_big.append(media_big_1)
    coeff_2_big.append(media_big_2)
    coeff_3_big.append(media_big_3)
    coeff_4_big.append(media_big_4)

In [ ]:
df = pd.DataFrame({
    'Small': coeff_1_small,
    'Mid': coeff_1_mid,
    'Big': coeff_1_big
})

df1 = pd.DataFrame({
    'Small': coeff_2_small,
    'Mid': coeff_2_mid,
    'Big': coeff_2_big
})

df2 = pd.DataFrame({
    'Small': coeff_3_small,
    'Mid': coeff_3_mid,
    'Big': coeff_3_big
})

df3 = pd.DataFrame({
    'Small': coeff_4_small,
    'Mid': coeff_4_mid,
    'Big': coeff_4_big
})

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(20, 6))

all_values = pd.concat([df.melt(value_vars=["Small", "Mid", "Big"])['value'],
                        df1.melt(value_vars=["Small", "Mid", "Big"])['value'],
                        df2.melt(value_vars=["Small", "Mid", "Big"])['value'],
                        df3.melt(value_vars=["Small", "Mid", "Big"])['value']])
y_min, y_max = all_values.min(), all_values.max()
margin = (y_max - y_min) * 0.25  
y_min, y_max = y_min - margin, y_max + margin

df_melted = df.melt(value_vars=["Small", "Mid", "Big"], 
                    var_name="Type", value_name="Value")

sns.violinplot(y='Value', hue='Type', data=df_melted, inner="quart", 
               palette="muted", ax=axes[0], alpha=0.75)
axes[0].set_title('Fattore 1 per Small, Mid e Big Cap')
axes[0].set_ylabel('Valore')
axes[0].set_ylim(y_min, y_max)  

df_melted = df1.melt(value_vars=["Small", "Mid", "Big"], 
                    var_name="Type", value_name="Value")

sns.violinplot(y='Value', hue='Type', data=df_melted, inner="quart", 
               palette="muted", ax=axes[1], alpha=0.75)
axes[1].set_title('Fattore 2 per Small, Mid e Big Cap')
axes[1].set_ylabel('Valore')
axes[1].set_ylim(y_min, y_max)  

df_melted = df2.melt(value_vars=["Small", "Mid", "Big"], 
                    var_name="Type", value_name="Value")

sns.violinplot(y='Value', hue='Type', data=df_melted, inner="quart", 
               palette="muted", ax=axes[2], alpha=0.75)
axes[2].set_title('Fattore 3 per Small, Mid e Big Cap')
axes[2].set_ylabel('Valore')
axes[2].set_ylim(y_min, y_max) 

df_melted = df3.melt(value_vars=["Small", "Mid", "Big"], 
                    var_name="Type", value_name="Value")

sns.violinplot(y='Value', hue='Type', data=df_melted, inner="quart", 
               palette="muted", ax=axes[3], alpha=0.75)
axes[3].set_title('Fattore 4 per Small, Mid e Big Cap')
axes[3].set_ylabel('Valore')
axes[3].set_ylim(y_min, y_max)  

plt.tight_layout()
plt.show()


# Modello Fama Macbeth

Salvataggio dei coefficienti da regression results.

In [ ]:
# Dizionario per memorizzare solo i coefficienti
coefficients_results = {}

for paese in paesi:
    coefficients_results[paese] = {}
    
    for categoria in categorie:
        for i in range(1, 10):
            portafoglio_key = f'{categoria}_portafoglio_{i}'
            
            coefficients_results[paese][portafoglio_key] = regression_results[paese][portafoglio_key]['coefficients']


## Regressioni

In [ ]:
# Dizionario per memorizzare i risultati delle regressioni Fama-MacBeth
fama_macbeth_results = {}

ols_significant = 0
ols_hc3_significant = 0
rlm_significant = 0

paesi = ['Germania', 'Francia', 'Italia', 'Spagna', 'Finlandia', 'Olanda']
categorie = ['SizeBM', 'SizeEbit', 'SizeINV']

num_periodi = 80  

# Per ogni trimestre
for periodo in range(num_periodi):
    y_values = []  
    x_values = [] 

    # Per ogni paese
    for paese in paesi:
        # Per ogni categoria
        for categoria in categorie:
            # Per ogni portafoglio
            for i in range(1, 10):  
                y_value = data_frames[paese][f'{categoria}_portafoglio_{i}']['y'][periodo]
                y_values.append(y_value)

                coefficients = coefficients_results[paese][f'{categoria}_portafoglio_{i}']
                x_values.append(coefficients[1:])  # Il primo coefficiente è escluso (intercetta)

    y_array = np.array(y_values)  # y diventa un array unidimensionale.
    X = pd.DataFrame(x_values)  # X diventa un DataFrame.

    if X.columns.duplicated().any():
        print(f"Attenzione: Colonne duplicate trovate per il periodo {periodo}. Controlla i dati.")
        continue

    # Aggiungi un'intercetta a X
    X = sm.add_constant(X)
    is_significant = False  


    # Esegui la regressione OLS 
    if len(y_array) == len(X):
        ols_model = sm.OLS(y_array, X).fit()

        # Test JB per la normalità dei residui.
        jb_statistic, jb_pvalue, _, _ = jarque_bera(ols_model.resid)

        # Se non normali eseguo RLM
        if jb_pvalue < 0.05:
            rlm_model = sm.RLM(y_array, X).fit()
            if ss_total <= 0:
                raise ValueError("La somma totale dei quadrati (SS_total) deve essere positiva.")

            rlm_predicted = rlm_model.predict(X)
            ss_total = np.sum((y - np.mean(y))**2) 
            ss_resid = np.sum(rlm_model.resid**2)
            if ss_total >= ss_resid:
                print("ss_total maggiore di ss_resid.")
            pseudo_r_squared = max(-1, 1 - (ss_resid / ss_total))

            # Controlla la significatività del modello RLM (basato sui p-value dei coefficienti)
            if rlm_model.pvalues.iloc[1:].min() < 0.05:  
                rlm_significant += 1
                is_significant = True
            
            coefficients = pd.Series(rlm_model.params[1:], index=X.columns[1:])
            pvalues = pd.Series(rlm_model.pvalues[1:], index=X.columns[1:])

            fama_macbeth_results[f'periodo_{periodo}'] = {
                'coefficients': coefficients,  
                'pvalues': pvalues,      
                'adj_r_squared': pseudo_r_squared,
                'is_significant': is_significant,
                'model_type': 'RLM'
            }
        else:
            # Test di White per eteroschedasticità
            white_test = het_white(ols_model.resid, ols_model.model.exog)
            white_pvalue = white_test[1]

            if white_pvalue < 0.05:
                # Se i residui sono eteroschedastici --> OLS HC3
                ols_hc3_model = ols_model.get_robustcov_results(cov_type='HC3')
                print(X.columns)
                coefficients_series = pd.Series(ols_hc3_model.params[1:], index=X.columns[1:])
                pvalues_series = pd.Series(ols_hc3_model.pvalues[1:], index=X.columns[1:])

                if ols_hc3_model.f_pvalue < 0.05:  
                    ols_hc3_significant += 1
                    is_significant = True

                fama_macbeth_results[f'periodo_{periodo}'] = {
                    'coefficients': coefficients_series,  
                    'pvalues': pvalues_series,            
                    'adj_r_squared': ols_hc3_model.rsquared,
                    'is_significant': is_significant,
                    'model_type': 'OLS HC3'
                }
            else:
                if ols_model.f_pvalue < 0.05:  
                    ols_significant += 1
                    is_significant = True
                # Se i residui sono normali e omoschedastici salvo i risultati del modello OLS.
                fama_macbeth_results[f'periodo_{periodo}'] = {
                    'coefficients': ols_model.params[1:],   
                    'pvalues': ols_model.pvalues[1:],       
                    'adj_r_squared': ols_model.rsquared,
                    'is_significant': is_significant,
                    'model_type': 'OLS'
                }


# Visualizza i risultati delle regressioni Fama-MacBeth
for key, result in fama_macbeth_results.items():
    print(f"{key}: Model Type: {result['model_type']}, Coefficients: {result['coefficients']}, "
          f"P-values: {result['pvalues']}, Adjusted R-squared: {result['adj_r_squared']}")


# Calcolo statistiche
ols_count = 0
ols_hc3_count = 0
rlm_count = 0

ols_r_squared = []
ols_hc3_r_squared = []
rlm_pseudo_r_squared = []


for result in fama_macbeth_results.values():
    model_type = result['model_type']
    adj_r_squared = result['adj_r_squared']
    pvalues = result['pvalues']

    if model_type == 'OLS':
        ols_count += 1
        ols_r_squared.append(adj_r_squared)
    elif model_type == 'OLS HC3':
        ols_hc3_count += 1
        ols_hc3_r_squared.append(adj_r_squared)
    elif model_type == 'RLM':
        rlm_count += 1
        rlm_pseudo_r_squared.append(adj_r_squared)

# Calcolo le medie degli R² e pseudo-R²
ols_avg_r_squared = sum(ols_r_squared) / len(ols_r_squared) if ols_r_squared else None
ols_hc3_avg_r_squared = sum(ols_hc3_r_squared) / len(ols_hc3_r_squared) if ols_hc3_r_squared else None
rlm_avg_pseudo_r_squared = sum(rlm_pseudo_r_squared) / len(rlm_pseudo_r_squared) if rlm_pseudo_r_squared else None

summary_df = pd.DataFrame({
    'Model Type': ['OLS', 'OLS HC3', 'RLM'],
    'Number of Regressions': [ols_count, ols_hc3_count, rlm_count],
    'Average R²': [ols_avg_r_squared, ols_hc3_avg_r_squared, None],
    'Average Pseudo R²': [None, None, rlm_avg_pseudo_r_squared],
    'Number of Significant Models': [ols_significant, ols_hc3_significant, rlm_significant]
})

# Salva i risultati in un file Excel
output_file = 'fama_macbeth_summary_SHOCK.xlsx'
summary_df.to_excel(output_file, index=False)

print(f"Risultati salvati in {output_file}")


# Lista per memorizzare i coefficienti di ogni variabile per ciascun periodo
coefficients_by_variable = {}

# Raccogli i coefficienti per ogni variabile attraverso i periodi
for periodo, result in fama_macbeth_results.items():
    coefficients = result['coefficients']
    for idx, coeff in enumerate(coefficients):
        if idx not in coefficients_by_variable:
            coefficients_by_variable[idx] = []
        coefficients_by_variable[idx].append(coeff)

# Calcola media e statistica t per ogni variabile
fama_macbeth_summary = {}
num_periodi_validi = len(fama_macbeth_results)

for idx, coeff_list in coefficients_by_variable.items():
    mean_coefficient = np.mean(coeff_list)
    
    std_dev = np.std(coeff_list, ddof=1)
    
    t_statistic = mean_coefficient / (std_dev / np.sqrt(num_periodi_validi))
    
    fama_macbeth_summary[f'variabile_{idx}'] = {
        'mean_coefficient': mean_coefficient,
        't_statistic': t_statistic,
        'std_dev': std_dev
    }

# Suddividi i periodi in gruppi da 8 e calcola media e t-statistica per ogni variabile
fama_macbeth_summary_8 = {}
num_groups = num_periodi // 8
if num_periodi % 8 != 0:
    num_groups += 1

for group in range(num_groups):
    start_period = group * 8
    end_period = min((group + 1) * 8, num_periodi)

    group_coefficients = {}
    for idx, coeff_list in coefficients_by_variable.items():
        group_coefficients[idx] = coeff_list[start_period:end_period]

    for idx, coeff_list in group_coefficients.items():
        mean_coefficient = np.mean(coeff_list)
        std_dev = np.std(coeff_list, ddof=1)
        t_statistic = mean_coefficient / (std_dev / np.sqrt(len(coeff_list)))

        fama_macbeth_summary_8[f'group_{group+1}_variabile_{idx}'] = {
            'mean_coefficient': mean_coefficient,
            't_statistic': t_statistic,
            'std_dev': std_dev
        }


## Visualizzazione coefficienti (Premi per il Rischio). 
Per ogni coefficiente si calcola la media annuale (dei 4 trimestri) e la signficatività di questo valore medio.

In [ ]:
mean_coefficients_per_group = []
t_statistics_per_group = []

num_groups = num_periodi // 4 # un istogramma per ogni anno (4 trimestri)

for group in range(num_groups):
    start_idx = group * 4
    end_idx = start_idx + 4

    group_coefficients = []
    group_t_statistics = []

    # Calcolo la media e la t-statistic per ciascuna variabile in questo gruppo
    for idx in coefficients_by_variable:
        coefficients_for_group = coefficients_by_variable[idx][start_idx:end_idx]
        
        mean_coefficient = np.mean(coefficients_for_group)
        std_dev = np.std(coefficients_for_group, ddof=1)
        
        t_statistic = mean_coefficient / (std_dev / np.sqrt(4)) # 4 è il nuemro di trimestri nel gruppo 

        group_coefficients.append(mean_coefficient)
        group_t_statistics.append(t_statistic)

    mean_coefficients_per_group.append(group_coefficients)
    t_statistics_per_group.append(group_t_statistics)

# Visualizzazione dei dati in due istrogrammi, uno per la media e uno per la statistica tmean_coefficients_per_group = np.array(mean_coefficients_per_group)
t_statistics_per_group = np.array(t_statistics_per_group)

group_labels = [f'{2003 + (i*1)}' for i in range(num_groups)] 

fig, ax = plt.subplots(2, 1, figsize=(10, 8), sharex=True)

bar_width = 0.2  
opacity = 0.8  

# Istogramma per le medie dei coefficienti
for i in range(mean_coefficients_per_group.shape[1]):
    ax[0].bar(np.array(range(num_groups)) + (i - 1) * bar_width, mean_coefficients_per_group[:, i],
              width=bar_width, alpha=opacity, label=f'Variabile {i+1}', edgecolor='black')

ax[0].set_title('Media dei Coefficienti per Ogni Gruppo di Periodi', fontsize=14, fontweight='bold')
ax[0].set_xlabel('Intervallo Temporale', fontsize=12)
ax[0].set_ylabel('Media del Coefficiente', fontsize=12)
ax[0].set_xticks(np.arange(num_groups))
ax[0].set_xticklabels(group_labels, rotation=45, ha="right", fontsize=10)
ax[0].grid(True, axis='y', linestyle='--', alpha=0.7)
ax[0].legend(title='Variabili', loc='upper left', fontsize=10)

# Istogramma per le t-statistics
for i in range(t_statistics_per_group.shape[1]):
    ax[1].bar(np.array(range(num_groups)) + (i - 1) * bar_width, t_statistics_per_group[:, i],
              width=bar_width, alpha=opacity, label=f'Variabile {i+1}', edgecolor='black')

ax[1].set_title('t-statistic per Ogni Gruppo di Periodi', fontsize=14, fontweight='bold')
ax[1].set_xlabel('Intervallo Temporale', fontsize=12)
ax[1].set_ylabel('t-statistic', fontsize=12)
ax[1].set_xticks(np.arange(num_groups))
ax[1].set_xticklabels(group_labels, rotation=45, ha="right", fontsize=10)
ax[1].grid(True, axis='y', linestyle='--', alpha=0.7)
ax[1].legend(title='Variabili', loc='upper left', fontsize=10)

plt.tight_layout()

plt.show()

#Calcolo e print del numero di gruppi significativi per ogni fattore
t_stat_significance_threshold = 1.96  # Soglia per il 95% di confidenza

significant_variables = {1: 0, 2: 0, 3: 0, 4: 0}  


for group_idx in range(num_groups):
    for var_idx in range(mean_coefficients_per_group.shape[1]):
        t_stat = t_statistics_per_group[group_idx, var_idx]
        
        
        if abs(t_stat) > t_stat_significance_threshold:
            significant_variables[var_idx + 1] += 1 

# Stampa il numero di gruppi significativi per ciascuna variabile
print("Variabili significative per ogni variabile:")
for variable, count in significant_variables.items():
    print(f"Variabile {variable}: {count} gruppi significativi")